In [1]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import pandas as pd
import os

HOUSING_PATH=os.path.join("datasets","housing")
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

h=load_housing_data()
h['income_cat']=pd.cut(h['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1,2,3,4,5])
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(h, h['income_cat']): #h['income_cat'] 분포를 반영하여 h를 계층 샘플링
    strat_train_set=h.loc[train_index]
    strat_test_set=h.loc[test_index]


# 레이블, 비레이블로 분리
- 타겟값에 예측 변수에 변형을 가할 때 같이 변형되면 안 되므로 

In [2]:
h = strat_train_set.drop("median_house_value", axis=1)
h_labels= strat_train_set["median_house_value"]

# 누락 데이터 처리

### 누락 데이터 처리방법
- 해당 데이터 제거(dropna)
- 전체 특성 삭제(drop)
- 어떤 값으로 채운다.(0, 평균, 중간값 등 fillna, sklearn.SimpleImputer)

In [3]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
# 중간값으로 채우려면 비 수치형 데이터들 제외해야함
h_num  = h.drop('ocean_proximity', axis=1)
# 모든 특성의 중간값을 계산(=이 또한 객체 학습)
imputer.fit(h_num)
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155,    3.     ])

In [4]:
#학습된 객체로 누락값을 중간값으로 대체
X= imputer.transform(h_num)
h_tr= pd.DataFrame(X, columns=h_num.columns, index= h_num.index)

# 텍스트형/범주형 데이터 처리

In [5]:
h_cat= h[['ocean_proximity']]
h_cat.head()
#임의의 텍스트가 아니라 카테고리(범주형)임을 알 수 있다.

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN


In [6]:
#카테고리를 숫자로 변환한다.
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
h_cat_1hot=encoder.fit(h_cat)

In [7]:
encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

# 모든 특성조합 필드 구성
- 커스텀 변환기를 만든다.
- sklearn method를 구현하면 바로 사용할 수 있다.
    - fit, transform, fit_transform(TransformerMixin 상속시 자동 생성)
    - get_params, set_params(hp 튜닝시 필요, BaseEstimtor 상속시 자동 생성)
- 합리적인 기본값을 셋팅하는 것이 좋다.

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self,add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household= X[:, rooms_ix]/X[:, households_ix]
        population_per_household= X[:, population_ix]/X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
            
attr_adder=CombinedAttributesAdder(add_bedrooms_per_room=False)
h_extra_attrs= attr_adder.transform(h.values)

# 특성 스케일링
### 정규화
- min-max scaling(MinMaxScaler)
- a~b(보통 a=0,b=1)범위 안에 들도록 이동하고 스케일을 조정함 
- 상한 하한이 특정값으로 기대되는 경우 유용

### 표준화
- StandardScaler
- 평균을 뺀 후(표준화 후 평균 0되는 이유) 표준편차로 나누어 분산이 1이 되도록 함
- 상한 하한이 없어 신경망같은 알고리즘에서는 문제가 될 수 있지만, 이상치의 영향을 덜 받음

# 변환 파이프라인
- Pipeline

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attr_adder', CombinedAttributesAdder()),
    ('std_scaler',StandardScaler()),
])

h_num_tr= num_pipeline.fit_transform(h_num)

- ColumnTransformer
    - 같은 변환 파이프라인을 거치는 열들 별로 구분해서 전달
    - ex. 범주형, 수치형 열 구분해서 변환할 때 등

In [10]:
from sklearn.compose import ColumnTransformer

num_attribs= list(h_num)
cat_attribs= ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

h_prepared= full_pipeline.fit_transform(h)

# 모델 선정
- 후보 모델 몇 개 시도
    - K-Fold CV로 과대과소적합도 평가
- 선정된 모델에 대해 hp튜닝
    - GridSearchCV, RandomizedSearch
- 앙상블
- 테스트 세트로 시스템 최종평가
    - 다소 훈련세트에 맞춰져 있으므로 실제 테스트시 약간 성능 떨어지는 것이 보편적

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

model= RandomForestRegressor()
model.fit(h_prepared, h_labels)

scores= cross_val_score(model, h_prepared, h_labels, scoring='neg_mean_squared_error', cv=10)
model_rmse = np.sqrt(-scores)
model_rmse 

array([51841.35909491, 49014.32571127, 46786.84540369, 51805.13255844,
       47325.81325209, 51835.41145091, 52241.41165022, 49528.08341686,
       48391.30963858, 53907.78650892])

In [12]:
model_rmse.mean()    

50267.74786858961

In [13]:
import joblib
#joblib.dump(model, "my_model.pkl")
#model= joblib.load('my_model.pkl')

# 하이퍼 파라미터 튜닝 / 중요 특성 걸러내기

In [14]:
from sklearn.model_selection import GridSearchCV

params=[
    {'n_estimators':[3, 10, 30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3, 10], 'max_features':[2,3,4] }
]
model= RandomForestRegressor()
gridCV= GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error',return_train_score=True)
gridCV.fit(h_prepared, h_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [15]:
feat_importances = gridCV.best_estimator_.feature_importances_
feat_importances
# 이를 바탕으로 중요하지않은 특성 빼고 최종 모델 훈련 및 평가

array([6.07180390e-02, 5.59831233e-02, 4.48940037e-02, 1.54268531e-02,
       1.55826414e-02, 1.56205690e-02, 1.54207065e-02, 2.99542941e-01,
       1.50982143e-01, 3.18393188e-02, 1.13402633e-01, 4.16355144e-02,
       5.60950789e-03, 1.27936751e-01, 1.03134820e-04, 1.90706272e-03,
       3.39505668e-03])

# 평가, 신뢰구간 얻기

In [16]:
model= gridCV.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared= full_pipeline.transform(X_test)
predict= model.predict(X_test_prepared)

mse= mean_squared_error(y_test, predict)
rmse= np.sqrt(mse)
rmse

48708.14442038564

In [17]:
from scipy import stats
confidence=0.95
squared_errors= (predict- y_test)**2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1,
                        loc= squared_errors.mean(),
                        scale= stats.sem(squared_errors)))

array([46695.54594715, 50640.82004126])